In [ ]:
http://bigdataexaminer.com/data-science/understanding-bias-and-variance-using-python/

# Luther Problem Statement
Source: https://github.com/thisismetis/nyc16_ds7/tree/master/projects/02-luther

Back story:

Using information we scrape from the web, can we build linear regression models from which we can learn about the movie industry?

Data:

acquisition: web scraping
storage: flat files
sources: Box Office Mojo, The Numbers, any other publicly available information
Skills:

basics of the web (requests, HTML, CSS, JavaScript)
web scraping
numpy and pandas
statsmodels, scikit-learn
Analysis:

linear regression
Deliverable/communication:

organized project repository
slide presentation
visual and oral communication in presentations
write-up of process and results
Design:

iterative design process
scoping
"MVP"s and building outward
More information:

We'll learn about web scraping using two popular tools - BeautifulSoup and Selenium. You'll have to know the very basics of HTML. We'll also be evolving the way we use IPython notebooks—during this project we'll begin to use the notebook as a development scratchpad, where we test things out through interactive scripting, but then solidify our work in python modules with reusable functions and classes.

We'll practice using linear regression. We'll have a first taste of feature selection, this time based on our intuition and some trial and error, and we'll build and refine our models.

We'll work in groups for brainstorming and design, and code sharing will be highly encouraged, but the final projects will be individual.

This project will really give you the freedom to challenge yourself, no matter your skill level. Find your boundaries, meet them, and push them a little further.

We are very excited to see what you will learn and do for Project Luther!

# Luther Project Summary Statement
Here I describe where I got my data. What my hypothesis is all about
Introduction explaining the study and source of the day. And what terms mean

# Background Information on Variables

http://research.microsoft.com/pubs/163535/FacebookPersonality_michal_29_04_12.pdf

# Import Data

In [27]:
import pandas as pd

In [28]:
!ls mypersonality_final/

celli-al_wcpr13.pdf     mypersonality_final.csv readme.txt


In [29]:
df_raw = pd.read_csv('mypersonality_final/mypersonality_final.csv')

In [30]:
my_dict = {"Joe":25, "Mary":23, "Bob":42}

In [31]:
df_raw.head()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [38]:
df_grouped = df_raw.groupby("#AUTHID", as_index = False)

In [39]:
print df_grouped.head()

                               #AUTHID  \
0     b7b7764cfa1c523e4e93ab2a79a946c4   
1     b7b7764cfa1c523e4e93ab2a79a946c4   
2     b7b7764cfa1c523e4e93ab2a79a946c4   
3     b7b7764cfa1c523e4e93ab2a79a946c4   
4     b7b7764cfa1c523e4e93ab2a79a946c4   
102   318bf822d4f2bd3920367560218619c0   
103   318bf822d4f2bd3920367560218619c0   
104   318bf822d4f2bd3920367560218619c0   
105   318bf822d4f2bd3920367560218619c0   
106   318bf822d4f2bd3920367560218619c0   
171   ecbddbfe00e0f83cfdb802a7186061c7   
172   ecbddbfe00e0f83cfdb802a7186061c7   
173   ecbddbfe00e0f83cfdb802a7186061c7   
174   ecbddbfe00e0f83cfdb802a7186061c7   
175   ecbddbfe00e0f83cfdb802a7186061c7   
236   db39f7b2aad360b1033ec1f8fcd57f9c   
237   db39f7b2aad360b1033ec1f8fcd57f9c   
238   db39f7b2aad360b1033ec1f8fcd57f9c   
239   db39f7b2aad360b1033ec1f8fcd57f9c   
240   db39f7b2aad360b1033ec1f8fcd57f9c   
246   4d035bd3fd8d9595d15cea9e388964be   
247   4d035bd3fd8d9595d15cea9e388964be   
248   4d035bd3fd8d9595d15cea9e3889

In [41]:
df_grouped.shape()

AttributeError: Cannot access attribute 'shape' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [34]:
df_grouped.head()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.00,3.15,3.25,4.40,n,y,n,n,y,06/19/09 03:21 PM,180,14861.60,93.29,0.03,15661,0.49,0.10
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.00,3.15,3.25,4.40,n,y,n,n,y,07/02/09 08:41 AM,180,14861.60,93.29,0.03,15661,0.49,0.10
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.00,3.15,3.25,4.40,n,y,n,n,y,06/15/09 01:15 PM,180,14861.60,93.29,0.03,15661,0.49,0.10
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3.00,3.15,3.25,4.40,n,y,n,n,y,06/22/09 04:48 AM,180,14861.60,93.29,0.03,15661,0.49,0.10
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3.00,3.15,3.25,4.40,n,y,n,n,y,07/20/09 02:31 AM,180,14861.60,93.29,0.03,15661,0.49,0.10
102,318bf822d4f2bd3920367560218619c0,has bed bugs..... ewwwww!,4.50,4.00,3.00,4.50,3.75,y,y,n,y,n,06/27/09 11:21 PM,318,49024.80,97.88,0.02,49584,0.49,0.06
103,318bf822d4f2bd3920367560218619c0,and mosquito bites,4.50,4.00,3.00,4.50,3.75,y,y,n,y,n,06/23/09 05:41 AM,318,49024.80,97.88,0.02,49584,0.49,0.06
104,318bf822d4f2bd3920367560218619c0,Mondays are the new thursdays- anthony bourdai...,4.50,4.00,3.00,4.50,3.75,y,y,n,y,n,06/23/09 05:41 AM,318,49024.80,97.88,0.02,49584,0.49,0.06
105,318bf822d4f2bd3920367560218619c0,is annoyed with herself,4.50,4.00,3.00,4.50,3.75,y,y,n,y,n,06/19/09 12:26 AM,318,49024.80,97.88,0.02,49584,0.49,0.06
106,318bf822d4f2bd3920367560218619c0,"is not turning on her cell at the moment, or f...",4.50,4.00,3.00,4.50,3.75,y,y,n,y,n,07/10/09 10:23 PM,318,49024.80,97.88,0.02,49584,0.49,0.06


# Clean Data as Necesesary

In [11]:
df_raw.describe()

,sEXT,sNEU,sAGR,sCON,sOPN,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
count,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9917.000000,9916.000000
mean,3.354760,2.609453,3.616643,3.474201,4.130386,429.377120,135425.315359,94.665170,3.154012,137642.476201,0.489920,0.128821
std,0.857578,0.760248,0.682485,0.737215,0.585672,428.760382,199433.803497,5.506696,311.073343,201392.066555,0.011908,0.106063
min,1.330000,1.250000,1.650000,1.450000,2.250000,24.000000,93.250000,0.040000,0.000000,0.490000,0.180000,0.000000
25%,2.710000,2.000000,3.140000,3.000000,3.750000,196.000000,16902.200000,93.770000,0.010000,17982.000000,0.490000,0.060000
50%,3.400000,2.600000,3.650000,3.400000,4.250000,317.000000,47166.900000,96.440000,0.020000,48683.000000,0.490000,0.090000
75%,4.000000,3.050000,4.150000,4.000000,4.550000,633.000000,196606.000000,97.880000,0.030000,198186.000000,0.500000,0.170000
max,5.000000,4.750000,5.000000,5.000000,5.000000,29724.900000,1251780.000000,99.820000,30978.000000,1263790.000000,0.500000,0.630000


In [25]:
df_raw.shape

(9917, 20)

In [15]:
# G
df_raw.head()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [26]:
df_raw.groupby(#AUTHID, idex = false)

SyntaxError: unexpected EOF while parsing (<ipython-input-26-ed8cdd3cf4a8>, line 1)

# Summary Statistics & Descriptive Visualizations (Pandas and Matplotlib, etc)

### Perform Scatter plots (y vs. Xi) for each of the X's -- this suggest relationship

### Check features for normality by using a density plot - plot normal on top of it. If it's not normal, then do some transformations on the features until they get to be normal

# Testing (Regression)

### Step 1a: (via intuition) Generate different models and provide explanations of why (avoid products, okay for squares and logs)

### Step 1b: Algorithmly - go throuhg all the options and create them.

### Step 2: Separate Data into 3 groupings: training, cv, and 
- Create seed

### Step 3: Using the Training set, determine the best coefficients for each model... (Fitting a model -- sm.fit()))

### Step 4: Against the CV set, compare the cost functions for each of the models and choose which model is the best (predict... and then get the difference) + ALso compare the adjusted R squared for all of them

### Step 5: Once the model is chosen, retrain that model using the combination of CV + Training

### Step 6: Test your final model against your test set.

In [ ]:
Diagnostic Tool #1:

https://theclevermachine.files.wordpress.com/2013/04/bias-variance-train-test-error.png

In [ ]:
Diagnostic Tool #2
### Step 7 - If you increase the size of your training set (from 0 to the full amount)
http://www.bigdataexaminer.com/wp-content/uploads/2014/11/code-9.png

# Analysis & Interpretations

# Next Steps

In [ ]:
Add sentiment analysis stuff
# Create Repository called luther (?)